# 可视化
## YinMo19

In [93]:
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mdates
import datetime

from scipy.integrate import odeint
from scipy import linalg as la
from scipy import optimize
import scipy
import csv
import pandas as pd

config = {
    "text.usetex": True,
    "text.latex.preamble": r"\usepackage{CJK}",  # 预先导入CJK宏包处理中文
}
plt.rcParams.update(config)

真实数据

In [105]:
# 数据加载
data = pd.read_csv("../../附件/附件1.csv", encoding="GB2312")
ALL_SC = list(set(data["分拣中心"]))
data["日期"] = pd.to_datetime(data["日期"])
data = data.sort_values("日期")
# 数据加载
pre_data = pd.read_csv("../SC_datas/predicted_volumes.csv", encoding="GB2312")
pre_data["date"] = pd.to_datetime(pre_data["date"])
pre_data = pre_data.sort_values("date")

In [95]:
data_for_sc = pd.read_csv("../../附件/附件1.csv", encoding="GB2312")
ALL_SC = list(set(data_for_sc["分拣中心"]))
existing_scs = list(map(lambda SC_:int(SC_[2:]),ALL_SC))
existing_scs.sort()

In [ ]:
data_example_3 = (
    data[data["分拣中心"] == "SC3"].groupby("日期")["货量"].sum().reset_index()
)
dates_3 = data_example_3["日期"]
values_3 = data_example_3["货量"]

dates_3 = np.array(dates_3)
values_3 = np.array(values_3)

# 仅使用SC48分拣中心的数据作为示例
pre_data_example = pre_data.groupby("date")["value"].sum().reset_index()
pre_dates = pre_data_example["date"]
pre_values = pre_data_example["value"]

pre_dates = np.array(pre_dates)
pre_values = np.array(pre_values)

with PdfPages(f"images/example.pdf") as pdf:
    plt.figure(figsize=(10, 6))
    plt.plot(dates_3, values_3, label=r"\begin{CJK*}{UTF8}{gbsn}SC6\end{CJK*}")
    plt.plot(pre_dates, pre_values, label=r"\begin{CJK*}{UTF8}{gbsn}SC41\end{CJK*}")
    plt.xlabel(r"\begin{CJK*}{UTF8}{gbsn}日期\end{CJK*}")
    plt.ylabel(r"\begin{CJK*}{UTF8}{gbsn}货量\end{CJK*}")
    plt.legend()
    plt.grid(True)

    pdf.savefig()
    plt.close()

In [96]:
data_example_6 = (
    data[data["分拣中心"] == "SC6"].groupby("日期")["货量"].sum().reset_index()
)
dates_6 = data_example_6["日期"]
values_6 = data_example_6["货量"]

dates_6 = np.array(dates_6)
values_6 = np.array(values_6)

data_example_41 = (
    data[data["分拣中心"] == "SC41"].groupby("日期")["货量"].sum().reset_index()
)
dates_41 = data_example_41["日期"]
values_41 = data_example_41["货量"]

dates_41 = np.array(dates_41)
values_41 = np.array(values_41)
with PdfPages(f"images/example.pdf") as pdf:
    plt.figure(figsize=(10, 6))
    plt.plot(dates_6, values_6, label=r"\begin{CJK*}{UTF8}{gbsn}SC6\end{CJK*}")
    plt.plot(dates_41, values_41, label=r"\begin{CJK*}{UTF8}{gbsn}SC41\end{CJK*}")
    plt.xlabel(r"\begin{CJK*}{UTF8}{gbsn}日期\end{CJK*}")
    plt.ylabel(r"\begin{CJK*}{UTF8}{gbsn}货量\end{CJK*}")
    plt.legend()
    plt.grid(True)

    pdf.savefig()
    plt.close()

In [97]:
for SC_example in ALL_SC:
    SC_example = int(SC_example[2:])
    data_example = (
        data[data["分拣中心"] == f"SC{SC_example}"]
        .groupby("日期")["货量"]
        .sum()
        .reset_index()
    )
    dates = data_example['日期']
    values = data_example['货量']

    dates = np.array(dates)
    values = np.array(values)

    # 仅使用SC48分拣中心的数据作为示例
    pre_data_example = (
        pre_data[pre_data["SC_id"] == SC_example]
        .groupby("date")["predicted_volume"]
        .sum()
        .reset_index()
    )
    pre_dates = pre_data_example["date"]
    pre_values = pre_data_example["predicted_volume"]

    pre_dates = np.array(pre_dates)
    pre_values = np.array(pre_values)

    with PdfPages(f"images/pre_{SC_example}.pdf") as pdf:
        plt.figure(figsize=(10, 6))
        plt.plot(dates, values,label=r"\begin{CJK*}{UTF8}{gbsn}真实数据\end{CJK*}")
        plt.plot(pre_dates, pre_values,label=r"\begin{CJK*}{UTF8}{gbsn}拟合数据\end{CJK*}")
        plt.xlabel(r"\begin{CJK*}{UTF8}{gbsn}日期\end{CJK*}")
        plt.ylabel(r"\begin{CJK*}{UTF8}{gbsn}货量\end{CJK*}")
        plt.legend()
        plt.grid(True)
        
        pdf.savefig()
        plt.close()

In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
from matplotlib.cm import get_cmap

# Suppose we have read the following data from a CSV file
data = pd.read_csv("../../附件/附件3.csv", encoding="GB2312")

df = pd.DataFrame(data)

# Assign angles for each unique center
centers = pd.concat([df["始发分拣中心"], df["到达分拣中心"]]).unique()
angle_map = dict(zip(centers, np.linspace(0, 2 * np.pi, len(centers), endpoint=False)))
color_map = get_cmap("viridis")

# Define control radius to avoid curves intersecting at the center
control_radius_adjusted = 0.5

# Line width and dot size
line_width = 1.0  # Thinner line
dot_size = 40  # Larger dot size

# Start plotting
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={"polar": True})
# ax.set_ylim(0, 1.2)
ax.axis("off")  # Turn off the axis
ax.grid(False)  # No grid

# Plot Bezier curves
for _, row in df.iterrows():
    start_angle = angle_map[row["始发分拣中心"]]
    end_angle = angle_map[row["到达分拣中心"]]
    alpha = row["货量"] / df["货量"].max()  # Adjust alpha based on weight
    color_idx = centers.tolist().index(row["始发分拣中心"]) / len(centers)
    start_color = color_map(color_idx)

    # Bezier control points
    control_angle = (start_angle + end_angle) / 2 + np.pi
    control_radius = control_radius_adjusted
    verts = [(start_angle, 1), (control_angle, control_radius), (end_angle, 1)]
    codes = [Path.MOVETO, Path.CURVE3, Path.CURVE3]
    path = Path(verts, codes)
    patch = patches.PathPatch(
        path, facecolor="none", lw=line_width, edgecolor=start_color, alpha=0.6
    )
    ax.add_patch(patch)

    # Add dots for each center
    ax.scatter(
        [start_angle], [1], color=start_color, s=dot_size, edgecolors="black", zorder=3
    )

# Add text labels
for center, angle in angle_map.items():
    color_idx = centers.tolist().index(center) / len(centers)
    ax.text(
        angle, 1.07, center, horizontalalignment="center", color=color_map(color_idx)
    )

with PdfPages("xuantu.pdf") as pdf:
    pdf.savefig()
    plt.close()

/tmp/ipykernel_28519/3589976955.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  color_map = get_cmap("viridis")
